In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [580]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
import bs4
from bs4 import BeautifulSoup
from pprint import pprint

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import os
import time

[nltk_data] Downloading package punkt to /Users/boogiedev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boogiedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [410]:
# Import Custom Modules
from src.helpers import *

In [25]:
url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421'

In [442]:
chrome_options = Options()  
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
page = driver.page_source
driver.close()
driver.quit()

In [443]:
html = BeautifulSoup(page, 'html.parser')

In [444]:
z = html.findAll('div', {'data-sephid':'5'})

In [339]:
# page = requests.get('https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421')
# html = BeautifulSoup(page.text, 'html.parser')

In [445]:
stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
stars

'4 stars'

In [446]:
star_dist = html.find('table', {'data-comp':'HistogramChart '})
star_counts = star_dist.get_text(separator=' ').split()[2::3]
star_dist = {f'{num}_stars':star_counts[i] for num, i in zip(range(5 ,0, -1), range(len(star_counts)))}

star_dist

{'5_stars': '2557',
 '4_stars': '539',
 '3_stars': '402',
 '2_stars': '402',
 '1_stars': '491'}

In [462]:
review_score = html.find("div", {'data-comp':"ReviewsStats Box "})

# review_score.get_text(separator='', strip=False)
# for i in review_score.findAll('div'):


# for x in review_score.findAll('td'):
#     print(x.get_text())
review_score.find('span').get_text()

'4391 reviews'

In [631]:
skin_types = ['normal', 'dry', 'oily', 'combination', 'sensitive']

test_url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421'
test_url = 'https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218'
test_url = 'https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639?icid2=products%20grid:p394639'
test_url = 'https://www.sephora.com/product/ultra-facial-cream-P421996?icid2=products%20grid:p421996'
page = requests.get(test_url)
html = BeautifulSoup(page.text, 'html.parser')
skin_content = html.find('div', {'data-at':'product_tabs_section'})
skin_type = skin_content.findAll('div', {'aria-labelledby':True})[0]



# print(x)
# isinstance(x, bs4.element.NavigableString)
# [e.extract() for e in skin_type.findAll('br')]
# x = skin_type.find('b', text=re.compile("skin", re.IGNORECASE))
# x.next_sibling

'Dry, Normal, and Combination '

In [310]:
test_url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421'
# test_url = 'https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218'
# test_url = 'https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639?icid2=products%20grid:p394639'
# test_url = 'https://www.sephora.com/product/ultra-facial-cream-P421996?icid2=products%20grid:p421996'
page = requests.get(test_url)
html = BeautifulSoup(page.text, 'html.parser')
skin_type = html.find('div', {'data-at':'product_tabs_section'})
ingredients = skin_type.findAll('div', {'aria-labelledby':True})[2]
splitted = ingredients.get_text(strip=False, separator=' |$%&*$| ')
x = splitted.split(' |$%&*$| ')
# x[3].split(', ')
parse_ingredients(x)

'Water/Aqua/Eau, Dicaprylyl Carbonate, Glycerin, Cetearyl Alcohol, Cetearyl Olivate, Sorbitan Olivate, Sclerocarya Birrea Seed Oil, Bacillus/Soybean/ Folic Acid Ferment Extract, Nymphaea Alba Root Extract, sh-Oligopeptide-1, sh-Oligopeptide-2, sh-Polypeptide-1, sh-Polypeptide-9, sh-Polypeptide-11, Copper Palmitoyl Heptapeptide-14, Heptapeptide-15 Palmitate, Palmitoyl Tetrapeptide-7, Palmitoyl Tripeptide-1, Alanine, Arginine, Glycine, Histidine, Isoleucine, Phenylalanine, Proline, Serine, Threonine, Valine, Acetyl Glutamine, Coconut Alkanes, Coco-Caprylate/Caprate, Sodium Hyaluronate, Aspartic Acid, Linoleic Acid, Linolenic Acid, Lecithin, Butylene Glycol, Polyvinyl Alcohol, Sodium Lactate, Sodium PCA, PCA, Sorbitan Isostearate, Carbomer, Polysorbate 20, Polysorbate 60, Lactic Acid/Glycolic Acid Copolymer, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Xanthan Gum, Isomalt, 1,2-Hexanediol, Caprylyl Glycol, Chlorphenesin, Phenoxyethanol, Tocopherol, Sodium Benzoate, Phe

In [634]:
# Parse for item features
def get_static_meta(item_url:str) -> dict:
    '''Returns item/product features from static section of given sephora url'''
    # GET Request to URL
    page = requests.get(item_url)
    # Create BS4 Object
    html = BeautifulSoup(page.text, "html.parser")
    
    # Find Name and Brand
    brand, name = (x.get_text() for x in html.find("h1", attrs={"data-comp":True}))
    
    # Find n_loves (product likes)
    n_loves = html.find('span', attrs={"data-at":"product_love_count"}).get_text()
    
    # Find Star Rating
    stars = html.find("div", {'data-comp':"StarRating "})['aria-label']
    
    # Find Ingredients and Skin Type Target
    tabbed_block = html.find('div', {'data-at':'product_tabs_section'})
    
    # Ingredients
    ingredient_section = tabbed_block.findAll('div', {'aria-labelledby':True})[2]
    ingredients = ingredient_section.get_text(strip=False, separator=' |$%&*$| ').split(' |$%&*$| ')
    ingredients = parse_ingredients(ingredients)
    # Skin Type
    skin_section = tabbed_block.findAll('div', {'aria-labelledby':True})[0]
    skin_types = parse_skin_type(skin_section)
    
    
    
    return brand, name, n_loves, stars, ingredients, skin_types

In [457]:
def get_dynamic_meta(url:str) -> tuple:
    '''Returns item/product features from scoll-loaded section of given sephora url'''
    # Init Chrome Driver Options
    chrome_options = Options()  
    # Run headless arg to avoid GUI
#     chrome_options.add_argument("--headless")
    # Init Driver Instance -> load options
    driver = webdriver.Chrome(options=chrome_options)
    
    # Call url via driver
    driver.get(url)
    # Scroll down to load REVEIW section
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    page = driver.page_source
    driver.close()
    driver.quit()
    
    # Create HTML object
    html = BeautifulSoup(page, 'html.parser')
    
    # Find n_reviews 
    n_reviews = html.find("div", {'data-comp':"ReviewsStats Box "}).find('span').get_text()
    
    # Find Star Distribution
    star_info = html.find('table', {'data-comp':'HistogramChart '})
    star_counts = star_info.get_text(separator=' ').split()[2::3]
    star_dist = [f'{num}_stars: {star_counts[i]}' for num, i in zip(range(5 ,0, -1), range(len(star_counts)))]
    
    return n_reviews, star_dist

In [632]:
def parse_ingredients(text_lists:list) -> list:
    '''Returns list of ingredients from a given text block, runs though pipeline to ensure only ingredients are taken'''
    stop_set = set(stopwords.words('english'))
    
    backup = ''
    for text in text_lists:
        set_i = set(text.lower().split())
        if text:
            if text.strip()[0] != '-':
                return text
            elif not (stop_set.intersection(set_i)):
                backup = text
    return backup


In [633]:
def parse_skin_type(text_block:bs4.element) -> str:
    '''Returns list of skin types from a given text block, runs though pipeline to ensure only valid types are taken'''
    html = text_block
    
    # Extract all break tags from text_block
    [e.extract() for e in html.findAll('br')]
    
    route = html.find('b', text=re.compile("skin", re.IGNORECASE))
    res = ''
    
    item = route.next_sibling
    while isinstance(item, bs4.element.NavigableString):
        res += item
        item = item.next_sibling
    return res
    
    
# parse_skin_type(skin_type)

In [635]:
def sephora_scrape(url:str) -> dict:
    '''Returns dictionary of needed features parsed from sephora.com link using selenium for dynamic page loading'''
    # Statically scrape product meta info
    brand, name, n_loves, stars, ingredients, skin_types = get_static_meta(url) 
    
    # Dynamically scrape product review section
    n_reviews, star_dist = get_dynamic_meta(url)
    
    return brand, name, n_loves, stars, ingredients, skin_types, n_reviews, star_dist

In [636]:
sephora_scrape('https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218')

('Tatcha',
 'The Water Cream',
 '232981',
 '4 stars',
 'Water, Saccharomyces/Camellia Sinensis Leaf/Cladosiphon Okamuranus/Rice Ferment Filtrate*, Dimethicone, Propanediol, Glycerin, Diglycerin, Diphenylsiloxy Phenyl Trimethicone, Gold, Belamcanda Chinensis Root Extract, Rosa Multiflora Fruit Extract, Houttuynia Cordata Extract, Sophora Angustifolia Root Extract, Sodium Hyaluronate, Lecithin, Pistacia Lentiscus (Mastic) Gum, Sodium Chloride, Sodium Citrate, Mica, Peg-9 Polydimethylsiloxyethyl Dimethicone, Dimethicone/Peg-10/15 Crosspolymer, Dimethicone/Phenyl Vinyl Dimethicone Crosspolymer, Disodium Edta, Tin Oxide, Titanium Dioxide, Butylene Glycol, Ethylhexylglycerin, Parfum/Fragrance, Alcohol, Phenoxyethanol. *Hadasei-3.\r\n',
 '✔ Normal✔ Oily✔ Combination✔  Sensitive\r\n',
 '3024 reviews',
 ['5_stars: 1815',
  '4_stars: 391',
  '3_stars: 306',
  '2_stars: 231',
  '1_stars: 281'])

In [637]:
sephora_scrape('https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639?icid2=products%20grid:p394639')

('belif',
 'The True Cream Aqua Bomb',
 '190974',
 '4.5 stars',
 'Water, Dipropylene Glycol, Glycerin, Methl Trimethicone, Alcohol Denat, Dimethicone, Cyclopentasiloxane, 1,2-Hexanediol, Malakite Extract, Caprylic/Capric Triglyceride, Pentaerythrityl Tetraethylhexanoate, PEG/PPG/Polybutylene Glycol-8/5/3 Glycerin, Alchemilla Vulgaris Leaf Extract*, Equisetum Arvense Leaf Extract*, Stellaria Media (Chickweed) Extract*, Urtica Dioica (Nettle) Leaf Extract*, Plantago Lanceolata Leaf Extract*, Avena Sativa (Oat) Kernel Extract**, Calendula Officinalis Flower Extract**, Nepeta Cataria Extract**, Rubus Idaeus (Raspberry) Leaf Extract**, Baptisia Tinctoria Root Extract**, Dimethiconol, Polymethylsilsesquioxane, Sodium Acrylate/Acryloyldimethyltaurate/Dimethylacrylamide Crosspolymer, Isohexadecane, Polysorbate 60, Ceramide 3, Cholesterol, Butyrospermum Parkii (Shea) Butter, Phenl Trimethicone, Pentaerythrityl Tetraisostearate, Panthenol, Squalane, Triethylhexanoin, Macadamia Ternifolia Seed Oi